In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [13]:
# Function to read the dataset.
def read_dataset():
    df = pd.read_csv("/home/ayush/Downloads/sonar.all-data.csv")
    #print(len(df.columns))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
     
    #Encode the dependent variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    y = one_hot_encode(y)
    print(X.shape)
    return (X,y)

In [10]:
# Definition Of Encoder Function.
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [14]:
# Read the Dataset.
X, Y = read_dataset()

# Shuffle the dataset to mixup the rows.
X, Y = shuffle(X, Y, random_state = 1)

# Breaking the dataset into testing and training part.
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0, random_state = 415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(207, 60)
(207, 60)
(207, 2)
(0, 60)


In [15]:
# Parameters to control the manipulation of our model accuracy and cost.
learning_rate = 0.3
training_epochs = 1000
cost_history = np.empty(shape = [1], dtype = float)
n_dim = X.shape[1]
print("Number of Dimensions ", n_dim)
n_class = 2
model_path = "home/NMI"

Number of Dimensions  60


In [16]:
# Definition of Neural Network
# Number of neurons
# Number of hidden layers
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

x = tf.placeholder(tf.float32, [None, n_dim])
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [None, n_class])

In [17]:
# Definition of Actual Model : MLP
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

In [19]:
# Weights and Biases

weights = {
    'h1' : tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2' : tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3' : tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4' : tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out' : tf.Variable(tf.truncated_normal([n_hidden_4, n_class]))    
}

biases = {
    'b1' : tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3' : tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4' : tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out' : tf.Variable(tf.truncated_normal([n_class]))
}

In [20]:
# Initialize the Variables
init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [28]:
# Calling the Model
y = multilayer_perceptron(x, weights, biases)

# Cost Function And Optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

sess = tf.Session()
sess.run(init)

mse_history = []
accuracy_history = []

for epoch in range(training_epochs):
    sess.run(training_step, feed_dict = {x: train_x, y_: train_y})
    cost = sess.run(cost_function, feed_dict = {x: train_x, y_: train_y})
    cost_history = np.append(cost_history, cost)
    correct_predicition = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predicition, tf.float32))
    #print("Accuracy: ", sess.run(accuracy, feed_dict = {x: test_x, y: test_y}))
    pred_y = sess.run(y, feed_dict = {x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict = {x: train_x, y_: train_y}))
    accuracy_history.append(accuracy)
    
    print('epoch : ', epoch, ' - ','cost : ', cost, " - MSE: ", mse_, "- Train Accuracy: ", accuracy)
    
save_path = saver.save(sess, model_path)
print("Model saved in file: %s" % save_path)

# Plotting MSE and Accuracy graph 

plt.plot(mse_history, 'r')
plt.show()
plt.plot(accuracy_history)
plt.show()

# Printing the final Accuracy

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict = {x: test_x, y:test_y})))

# Printing the final MSE

pred_y = sess.run(y, feed_dict = {x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))


epoch :  0  -  cost :  89.563286  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  1  -  cost :  10.88652  - MSE:  nan - Train Accuracy:  0.46376812
epoch :  2  -  cost :  13.3107  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  3  -  cost :  0.848984  - MSE:  nan - Train Accuracy:  0.46376812
epoch :  4  -  cost :  0.7541836  - MSE:  nan - Train Accuracy:  0.46376812
epoch :  5  -  cost :  0.7047248  - MSE:  nan - Train Accuracy:  0.46376812
epoch :  6  -  cost :  0.69067776  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  7  -  cost :  0.6896614  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  8  -  cost :  0.68932617  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  9  -  cost :  0.68903065  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  10  -  cost :  0.68877774  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  11  -  cost :  0.6885389  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  12  -  cost :  0.68830156  - MSE:  nan - Train Accuracy:  0.5362319
epoch :  13  -  c

epoch :  108  -  cost :  0.6762797  - MSE:  nan - Train Accuracy:  0.5748792
epoch :  109  -  cost :  0.66721994  - MSE:  nan - Train Accuracy:  0.6038647
epoch :  110  -  cost :  0.6671343  - MSE:  nan - Train Accuracy:  0.57004833
epoch :  111  -  cost :  0.6588081  - MSE:  nan - Train Accuracy:  0.6425121
epoch :  112  -  cost :  0.6605825  - MSE:  nan - Train Accuracy:  0.5603865
epoch :  113  -  cost :  0.6572778  - MSE:  nan - Train Accuracy:  0.65217394
epoch :  114  -  cost :  0.6610882  - MSE:  nan - Train Accuracy:  0.57004833
epoch :  115  -  cost :  0.6607282  - MSE:  nan - Train Accuracy:  0.62801933
epoch :  116  -  cost :  0.6659016  - MSE:  nan - Train Accuracy:  0.57004833
epoch :  117  -  cost :  0.6643543  - MSE:  nan - Train Accuracy:  0.6135266
epoch :  118  -  cost :  0.66917557  - MSE:  nan - Train Accuracy:  0.57004833
epoch :  119  -  cost :  0.66216063  - MSE:  nan - Train Accuracy:  0.6183575
epoch :  120  -  cost :  0.6647118  - MSE:  nan - Train Accuracy:  

epoch :  214  -  cost :  0.68435067  - MSE:  nan - Train Accuracy:  0.5845411
epoch :  215  -  cost :  0.6279871  - MSE:  nan - Train Accuracy:  0.65217394
epoch :  216  -  cost :  0.60513407  - MSE:  nan - Train Accuracy:  0.6956522
epoch :  217  -  cost :  0.595772  - MSE:  nan - Train Accuracy:  0.7198068
epoch :  218  -  cost :  0.5922519  - MSE:  nan - Train Accuracy:  0.71014494
epoch :  219  -  cost :  0.5924761  - MSE:  nan - Train Accuracy:  0.6956522
epoch :  220  -  cost :  0.599041  - MSE:  nan - Train Accuracy:  0.6763285
epoch :  221  -  cost :  0.62285537  - MSE:  nan - Train Accuracy:  0.62801933
epoch :  222  -  cost :  0.6917948  - MSE:  nan - Train Accuracy:  0.5942029
epoch :  223  -  cost :  0.6835433  - MSE:  nan - Train Accuracy:  0.5748792
epoch :  224  -  cost :  0.6574566  - MSE:  nan - Train Accuracy:  0.589372
epoch :  225  -  cost :  0.6160414  - MSE:  nan - Train Accuracy:  0.705314
epoch :  226  -  cost :  0.59888804  - MSE:  nan - Train Accuracy:  0.7053

epoch :  320  -  cost :  0.45922974  - MSE:  nan - Train Accuracy:  0.8019324
epoch :  321  -  cost :  0.512841  - MSE:  nan - Train Accuracy:  0.7198068
epoch :  322  -  cost :  0.8769333  - MSE:  nan - Train Accuracy:  0.589372
epoch :  323  -  cost :  0.8457152  - MSE:  nan - Train Accuracy:  0.4830918
epoch :  324  -  cost :  0.6031923  - MSE:  nan - Train Accuracy:  0.7826087
epoch :  325  -  cost :  0.5546482  - MSE:  nan - Train Accuracy:  0.7487923
epoch :  326  -  cost :  0.54439604  - MSE:  nan - Train Accuracy:  0.76328504
epoch :  327  -  cost :  0.5351209  - MSE:  nan - Train Accuracy:  0.7826087
epoch :  328  -  cost :  0.52582324  - MSE:  nan - Train Accuracy:  0.79227054
epoch :  329  -  cost :  0.5164641  - MSE:  nan - Train Accuracy:  0.7874396
epoch :  330  -  cost :  0.50769985  - MSE:  nan - Train Accuracy:  0.7874396
epoch :  331  -  cost :  0.4963257  - MSE:  nan - Train Accuracy:  0.8115942
epoch :  332  -  cost :  0.4813656  - MSE:  nan - Train Accuracy:  0.792

epoch :  426  -  cost :  0.4041857  - MSE:  nan - Train Accuracy:  0.79227054
epoch :  427  -  cost :  0.5926419  - MSE:  nan - Train Accuracy:  0.6376812
epoch :  428  -  cost :  0.41272157  - MSE:  nan - Train Accuracy:  0.8067633
epoch :  429  -  cost :  0.47026983  - MSE:  nan - Train Accuracy:  0.73913044
epoch :  430  -  cost :  0.4032717  - MSE:  nan - Train Accuracy:  0.79710144
epoch :  431  -  cost :  0.517997  - MSE:  nan - Train Accuracy:  0.6714976
epoch :  432  -  cost :  0.4312184  - MSE:  nan - Train Accuracy:  0.7874396
epoch :  433  -  cost :  0.53202516  - MSE:  nan - Train Accuracy:  0.65217394
epoch :  434  -  cost :  0.36880574  - MSE:  nan - Train Accuracy:  0.8357488
epoch :  435  -  cost :  0.36088493  - MSE:  nan - Train Accuracy:  0.85024154
epoch :  436  -  cost :  0.4003542  - MSE:  nan - Train Accuracy:  0.8019324
epoch :  437  -  cost :  0.6421729  - MSE:  nan - Train Accuracy:  0.6376812
epoch :  438  -  cost :  0.5601557  - MSE:  nan - Train Accuracy:  

epoch :  532  -  cost :  0.61002415  - MSE:  nan - Train Accuracy:  0.59903383
epoch :  533  -  cost :  0.49953455  - MSE:  nan - Train Accuracy:  0.7004831
epoch :  534  -  cost :  0.38973537  - MSE:  nan - Train Accuracy:  0.87922704
epoch :  535  -  cost :  0.32026455  - MSE:  nan - Train Accuracy:  0.8599034
epoch :  536  -  cost :  0.28610855  - MSE:  nan - Train Accuracy:  0.87922704
epoch :  537  -  cost :  0.2637142  - MSE:  nan - Train Accuracy:  0.8888889
epoch :  538  -  cost :  0.24990638  - MSE:  nan - Train Accuracy:  0.8937198
epoch :  539  -  cost :  0.24302664  - MSE:  nan - Train Accuracy:  0.8937198
epoch :  540  -  cost :  0.23768124  - MSE:  nan - Train Accuracy:  0.8937198
epoch :  541  -  cost :  0.2342682  - MSE:  nan - Train Accuracy:  0.90821254
epoch :  542  -  cost :  0.23271294  - MSE:  nan - Train Accuracy:  0.8888889
epoch :  543  -  cost :  0.24761197  - MSE:  nan - Train Accuracy:  0.8937198
epoch :  544  -  cost :  0.3762771  - MSE:  nan - Train Accura

epoch :  638  -  cost :  0.70780814  - MSE:  nan - Train Accuracy:  0.5652174
epoch :  639  -  cost :  0.44427595  - MSE:  nan - Train Accuracy:  0.7777778
epoch :  640  -  cost :  0.32449865  - MSE:  nan - Train Accuracy:  0.87922704
epoch :  641  -  cost :  0.26869965  - MSE:  nan - Train Accuracy:  0.90821254
epoch :  642  -  cost :  0.24259916  - MSE:  nan - Train Accuracy:  0.93719804
epoch :  643  -  cost :  0.22424014  - MSE:  nan - Train Accuracy:  0.9227053
epoch :  644  -  cost :  0.21379045  - MSE:  nan - Train Accuracy:  0.93236715
epoch :  645  -  cost :  0.20504865  - MSE:  nan - Train Accuracy:  0.90821254
epoch :  646  -  cost :  0.2040856  - MSE:  nan - Train Accuracy:  0.93236715
epoch :  647  -  cost :  0.22531438  - MSE:  nan - Train Accuracy:  0.884058
epoch :  648  -  cost :  0.32926023  - MSE:  nan - Train Accuracy:  0.84541065
epoch :  649  -  cost :  0.6639233  - MSE:  nan - Train Accuracy:  0.7536232
epoch :  650  -  cost :  0.81552476  - MSE:  nan - Train Acc

epoch :  743  -  cost :  0.6259851  - MSE:  nan - Train Accuracy:  0.76328504
epoch :  744  -  cost :  0.56891847  - MSE:  nan - Train Accuracy:  0.6666667
epoch :  745  -  cost :  0.32875112  - MSE:  nan - Train Accuracy:  0.8357488
epoch :  746  -  cost :  0.22234346  - MSE:  nan - Train Accuracy:  0.93719804
epoch :  747  -  cost :  0.19228818  - MSE:  nan - Train Accuracy:  0.9516908
epoch :  748  -  cost :  0.17952381  - MSE:  nan - Train Accuracy:  0.93236715
epoch :  749  -  cost :  0.17105411  - MSE:  nan - Train Accuracy:  0.9468599
epoch :  750  -  cost :  0.16659468  - MSE:  nan - Train Accuracy:  0.92753625
epoch :  751  -  cost :  0.16345204  - MSE:  nan - Train Accuracy:  0.95652175
epoch :  752  -  cost :  0.16900875  - MSE:  nan - Train Accuracy:  0.9130435
epoch :  753  -  cost :  0.18408139  - MSE:  nan - Train Accuracy:  0.93719804
epoch :  754  -  cost :  0.24546988  - MSE:  nan - Train Accuracy:  0.87922704
epoch :  755  -  cost :  0.40840423  - MSE:  nan - Train A

KeyboardInterrupt: 